# Predicting Car Battery Failure

In this notebook you will use compute resources provided by Azure Machine Learning (AML) to *remotely* train a set of models using Automated Machine Learning, evaluate performance of each model and pick the best performing model to deploy as a web service.

You will provision all Azure resources directly from this notebook, without having to use the Azure Portal to create any resources.

## Setup
To begin, you will need to provide the following information about your Azure Subscription. 

In the following cell, you retrieve or create the AML Workspace and then train one instance of the model that we will deploy. In this step, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments. Execute the following cell by selecting the `>|Run` button in the command bar above.

In [ ]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "e223f1b3-d19b-4cfa-98e9-bc9be62717bc"

#Provide values for the new Resource Group and Workspace that will be created
resource_group = "aml-workspace"
workspace_name = "aml-workspace"

#Optionally, set the Azure Region in which to deploy your Azure Machine Learning Workspace
workspace_region = "westcentralus" # other options include eastus, westcentralus, southeastasia, australiaeast, westeurope

In [25]:
# constants
experiment_name = 'automl-regression'
project_folder = './automl-regression'
data_url = "https://devintersection.blob.core.windows.net/data/simulated-data.csv"
cluster_name = "cpucluster"
aks_cluster_name = 'my-aks-cluster' 
resource_id = '/subscriptions/2a779d6f-0806-4359-a6e8-f1fd57bb5dd7/resourceGroups/devintersection-2018-aml-demo/providers/Microsoft.BatchAI/workspaces/devintersection-workspace/clusters/cpucluster1c848275bca'

### Import required packages

The Azure Machine Learning SDK provides a comprehensive set of a capabilities that you can use directly within a notebook including:
- Creating a **Workspace** that acts as the root object to organize all artifacts and resources used by Azure Machine Learning.
- Creating **Experiments** in your Workspace that capture versions of the trained model along with any desired model performance telemetry. Each time you train a model and evaluate its results, you can capture that run (model and telemetry) within an Experiment.
- Creating **Compute** resources that can be used to scale out model training, so that while your notebook may be running in a lightweight container in Azure Notebooks, your model training can actually occur on a powerful cluster that can provide large amounts of memory, CPU or GPU. 
- Using **Automated Machine Learning (AutoML)** to automatically train multiple versions of a model using a mix of different ways to prepare the data and different algorithms and hyperparameters (algorithm settings) in search of the model that performs best according to a performance metric that you specify. 
- Packaging a Docker **Image** that contains everything your trained model needs for scoring (prediction) in order to run as a web service.
- Deploying your Image to either Azure Kubernetes or Azure Container Instances, effectively hosting the **Web Service**.

In Azure Notebooks, all of the libraries needed for Azure Machine Learning are pre-installed. To use them, you just need to import them. Run the following cell to do so:

In [26]:
import logging
import os
import random
import re

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun


## Create and connect to an Azure Machine Learning Workspace

Run the following cell to create a new Azure Machine Learning Workspace and save the configuration to disk (next to the Jupyter notebook).

In [27]:
from azureml.core import Workspace

# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace instead of an error
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Library configuration succeeded')


Wrote the config file config.json to: /home/nbuser/library/aml_config/config.json
Library configuration succeeded


## Create a Workspace Experiment

Next, create a new Experiment.

In [4]:
ws = Workspace.from_config()

import os

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

experiment = Experiment(ws, experiment_name)

Found the config file in: /home/nbuser/library/aml_config/config.json


### Opt-in to send Diagnostics
During the Experiment, you can opt-in to automatically collecting diagnostic telemetry. Run the following cell to do so.

In [40]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

## Get and explore the Car Telemetry Data

Run the following cell to download and examine the car telemetry data. The model we want to build will try to predict how many days until the battery has a freeze event. Which fields do you think will be useful?

In [10]:
data = pd.read_csv(data_url)
print(data)

      Survival_In_Days  Suffered_Freeze_Event     Sex  Car_Is_Garaged State  \
0     975               False                  Female  False           KS     
1     1690              False                  Male    True            AR     
2     926               False                  Female  True            DE     
3     1742              False                  Male    True            IL     
4     1427              False                  Female  True            NY     
5     2280              False                  Male    False           CO     
6     743               False                  Female  True            HI     
7     1118              False                  Male    True            KY     
8     1835              False                  Female  True            AZ     
9     1508              False                  Male    False           FL     
10    1494              False                  Female  True            IN     
11    1511              False                  Male 

## Remotely train multiple models using Auto ML and Azure ML Compute

In the following cells, you will not train the model against the data you just downloaded using the resources provided by Azure Notebooks. Instead, you will deploy an Azure ML Compute cluster that will download the data and use Auto ML to train multiple models, evaluate the performance and allow you to retrieve the best model that was trained. As you will see this is almost entirely done thru configuration, with very little code required. 

### Create get_data.py for remote compute

The Azure Machine Learning Compute cluster needs to know how to get the data to train against. You can package this logic in a script that will be executed by the compute when it starts executing the training.

Run the following cells to locally create the get_data.py script that will be deployed to remote compute.

In [41]:
# create project folder
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

In [42]:
%%writefile $project_folder/get_data.py

import pandas as pd
import numpy as np

def get_data():
    
    data = pd.read_csv("https://devintersection.blob.core.windows.net/data/simulated-data.csv")
    
    X = data.iloc[:,4:72]
    Y = np.empty(data.shape[0], dtype=object)
    Y[:] = data.iloc[:,0].tolist()

    return { "X" : X, "y" : Y }

Overwriting ./automl-regression/get_data.py


### Create AML Compute Cluster

Now you are ready to create the compute cluster. Run the following cell to create a new compute cluster (or retrieve the existing cluster if it already exists). The code below will create a CPU based cluster where each node in the cluster is of the size `STANDARD_D12_V2`, and the cluster will have at most 4 such nodes. 

In [14]:
### Create AML CPU based Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

Found existing compute target.
{'allocationState': 'steady', 'allocationStateTransitionTime': '2018-12-04T18:13:30.463000+00:00', 'creationTime': '2018-12-04T18:13:10.187000+00:00', 'currentNodeCount': 0, 'errors': None, 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'succeeded', 'provisioningStateTransitionTime': '2018-12-04T18:13:29.797000+00:00', 'scaleSettings': {'manual': None, 'autoScale': {'maximumNodeCount': 4, 'minimumNodeCount': 0, 'initialNodeCount': 0}}, 'vmPriority': 'dedicated', 'vmSize': 'STANDARD_D12_V2'}


### Instantiate an Automated ML Config

Once the cluster is ready, you are ready to submit an Auto ML experiment run. This will remotely train multiple models, evaluate them and allow you review the performance characteristics of each one, as well as to pick the *best model* that was trained and download it. 

Run the following cell to configure the Auto ML run. In short what you are configuring here is the training of a Linear Regression model that will attempt to predict the value of the first feature (`Survival_in_days`) based on all the other features in the data set. The run is configured to try at most 100 iterations where no iteration can run longer that 10 minutes. Additionally, the data will be automatically pre-processed in different ways as a part of the automated model training (as indicated by the `preprocess` attribute having a value of `True`. The goal of Auto ML in this case is to find the best models that result, as measure by the R2 Score metric (as indicated by the `primary_metric` attribute). The local path to the script you created to retrieve the data is supplied to the AutoMLConfig, ensuring the file is made available to the remote cluster. The actual execution of this training will occur on the compute cluster you created previously. 

In general, the AutoMLConfig is very flexible, allowing you to specify all of the following:
- Task type (classification, regression, forecasting)
- Number of algorithm iterations and maximum time per iteration
- Accuracy metric to optimize
- Algorithms to blacklist/whitelist
- Number of cross-validations
- Compute targets
- Training data

In [19]:
automl_config = AutoMLConfig(task = 'regression',
                             iterations = 100,
                             iteration_timeout_minutes = 10, 
                             max_cores_per_iteration = 10,
                             preprocess= True,
                             primary_metric='r2_score',
                             n_cross_validations = 5,
                             debug_log = 'automl.log',
                             verbosity = logging.INFO,
                             data_script = project_folder + "/get_data.py",
                             compute_target = compute_target,
                             blacklist_models = "",
                             path = project_folder)

### Run our Experiment on AML Compute

With the configuration ready, run the following cell execute the experiment on the remote compute cluster.

In [20]:
remote_run = experiment.submit(automl_config, show_output=False)
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-regression,AutoML_99de1c5a-78e3-4f33-ac20-36e4b02eec20,automl,Preparing,Link to Azure Portal,Link to Documentation


### Display Workspace Experiments

Using the Azure Machine Learning SDK, you retrieve any of the experiments in your Workspace and drill into the details of any runs the experiment contains. Run the following cell to explore the number of runs by experiment name.

In [5]:
ws = Workspace.from_config()
experiment_list = Experiment.list(workspace=ws)

summary_df = pd.DataFrame(index = ['No of Runs'])
pattern = re.compile('^AutoML_[^_]*$')
for experiment in experiment_list:
    all_runs = list(experiment.get_runs())
    automl_runs = []
    for run in all_runs:
        if(pattern.match(run.id)):
            automl_runs.append(run)    
    summary_df[experiment.name] = [len(automl_runs)]
    
pd.set_option('display.max_colwidth', -1)
summary_df.T

Found the config file in: /home/nbuser/library/aml_config/config.json


,No of Runs
automl-regression,5


### Display Automated ML Runs for the Experiment

Similarly, you can view all of the runs that ran supporting Auto ML, view their status and see how each run performed against the configured performance metric.

In [6]:
proj = ws.experiments[experiment_name]
summary_df = pd.DataFrame(index = ['Type', 'Status', 'Primary Metric', 'Iterations', 'Compute', 'Name'])
pattern = re.compile('^AutoML_[^_]*$')
all_runs = list(proj.get_runs(properties={'azureml.runsource': 'automl'}))
for run in all_runs:
    if(pattern.match(run.id)):
        properties = run.get_properties()
        tags = run.get_tags()
        amlsettings = eval(properties['RawAMLSettingsString'])
        if 'iterations' in tags:
            iterations = tags['iterations']
        else:
            iterations = properties['num_iterations']
        summary_df[run.id] = [amlsettings['task_type'], run.get_details()['status'], properties['primary_metric'], iterations, properties['target'], amlsettings['name']]
    
from IPython.display import HTML
projname_html = HTML("<h3>{}</h3>".format(proj.name))

from IPython.display import display
display(projname_html)
display(summary_df.T)

,Type,Status,Primary Metric,Iterations,Compute,Name
AutoML_ab755820-4bfd-4e8a-8b4b-9e0a2446b1c2,regression,Completed,r2_score,100,d12-cluster,automl-regression
AutoML_e73345b0-5094-4bbe-970b-06f4521da5dd,regression,Completed,r2_score,10,d12-cluster,automl-regression
AutoML_1e6dd094-9885-4998-a2d0-0a598956ed65,regression,Completed,r2_score,10,d12-cluster,automl-regression
AutoML_3e948b39-bab0-4cd5-bb60-fb3bdcc79e7e,regression,Completed,r2_score,10,d12-cluster,automl-regression
AutoML_3ce57e56-58bb-45ab-923a-aa10393a4837,regression,NotStarted,r2_score,10,d12-cluster,automl-regression


### TODO: Display Automated ML Run Details

In [15]:
run_id = 'AutoML_ab755820-4bfd-4e8a-8b4b-9e0a2446b1c2' 

from azureml.widgets import RunDetails

experiment = Experiment(ws, experiment_name)
ml_run = AutoMLRun(experiment=experiment, run_id=run_id)

RunDetails(ml_run).show() 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET', …

### Show best run and model

At this point you have multiple runs, each with a different trained model. How can you get the model that performed the best? Run the following cells to learn how.

In [ ]:
best_run, fitted_model = ml_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
# show run and model from metric
lookup_metric = "root_mean_squared_error"
best_run, fitted_model = ml_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

In [ ]:
# show run and model from iteration 3
iteration = 3
third_run, third_model = ml_run.get_output(iteration=iteration)
print(third_run)
print(third_model)

At this point you now have a model you could use for predicting the time until battery failure. You would typically use this model in one of two ways:
- Use the model file within other notebooks to batch score predictions.
- Deploy the model file as a web service that applications can call. 

In the following, you will explore the latter option to deploy the best model as a web service.

## Deploy the Model as a Web Service

Azure Machine Learning provides a Model Registry that acts like a version controlled repository for each of your trained models. To version a model, you use  the SDK as follows. Run the following cell to register the model with Azure Machine Learning. 

In [49]:
# register the model for deployment
model = Model.register(model_path = "model.pkl",
                       model_name = "model.pkl",
                       tags = {'area': "auto", 'type': "regression"},
                       description = "Contoso Auto model to predict battery failure",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model model.pkl
model.pkl Contoso Auto model to predict battery failure 4


Once you have a model added to the registry in this way, you can deploy web services that pull their model directly from this repository when they first start up.

### Create Scoring File

Azure Machine Learning SDK gives you control over the logic of the web service, so that you can define how it retrieves the model and how the model is used for scoring. This is an important bit of flexibility. For example, you often have to prepare any input data before sending to your model. You can define this data preparation logic (as well as the model loading approach) in the scoring file. 

Run the following cell to create a scoring file that will be included in the Docker Image that contains your deployed web service.

In [79]:
%%writefile score.py
import pickle
import json
import numpy
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('model.pkl') # this name is model.id of model that we want to deploy
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

def run(rawdata):
    try:
        data = json.loads(rawdata)['data']
        data = numpy.array(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()})

Overwriting score.py


### Create Environment Dependency File

When you deploy a model as web service to either Azure Container Instance or Azure Kubernetes Service, you are deploying a Docker container. The first steps towards deploying involve defining the contents of that container. In the following cell, you create Conda Dependencies YAML file that describes what Python packages need to be installed in the container- in this case you specify scikit-learn, numpy and the Azure ML SDK. 

Execute the following cell.

In [59]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'],pip_packages=['azureml-sdk[notebooks,automl]'])
print(myenv.serialize_to_string())

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-sdk[notebooks,automl]==0.1.80
- numpy
- scikit-learn



### Create Container Image

To create a Container Image, you need three things: the scoring script file, the runtime configuration (defining whether Python or PySpark should be used) and the Conda Dependencies file. Calling `ContainerImage.image_configuration` will capture all of the container image configuration in a single object. This Image will be stored in an instance of Azure Container Registry that is associated with your Azure Machine Learning Workspace. 

Execute the following cell.

In [80]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "Image with regression model",
                                                  tags = {'area': "auto", 'type': "regression"}
                                                 )

image = ContainerImage.create(name = "myimage1",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running..................................................
SucceededImage creation operation finished for image myimage1:11, operation "Succeeded"


### Create AKS Compute Cluster

With the Container Image configuration in hand, you are almost ready to deploy to AKS.The first step is to create your AKS cluster. 

Execute the following cell to provision a small AKS cluster. This step will take about **15-20 minutes**.

In [18]:
try:
    # attach to existing cluster
    aks_target = AksCompute.attach_configuration(resource_group,aks_cluster_name)
    print('Found existing compute target.')

except ComputeTargetException:
    print('Creating a new compute target...')
    
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration()
    
    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)
    
    aks_target.wait_for_completion(True)

Found existing compute target.


In [63]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
CPU times: user 132 ms, sys: 19.2 ms, total: 151 ms
Wall time: 1.77 s


### Activate Data Collection and App Insights

In [34]:
aks_config = AksWebservice.deploy_configuration(collect_model_data=True, enable_app_insights=True)

### Deploy AKS Service

Now you are ready to deploy your web service to the AKS cluster. To deploy the container that operationalizes your model as a webservice, you can use `Webservice.deploy_from_image` which will use your registered Docker Image, pulling it from the Container Registry, and run the created container in AKS. 

Execute the following cell to deploy your webservice to AKS. This step will take **10-15 minutes** to complete.

In [81]:
%%time
aks_service_name ='aks-automl-service'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target
                                           )
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running..........
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 2.91 s, sys: 1.99 s, total: 4.91 s
Wall time: 1min 35s


### Test the deployed web service

With the deployed web service ready, you are now ready to test calling it with some car telemetry to see the scored results. There are three ways to approach this:
- You could use the `aks_service` object that you acquired in the previous cell to call the service directly.
- You could use the `Webservice` class to get a reference to a deployed web service by name.
- You could use any client capable of making a REST call.

In this notebook, we will take the second approach. Run the following cells to retrieve the web service by name and then to invoke it using some sample car telemetry.

In [33]:
# connect to our deployed webservice
aks_service_name ='aml-deployment'
aks_service = Webservice(ws,aks_service_name)

In [44]:
%%time
import json

test_sample = json.dumps({'data': [
['IN',19.15495,6.384985,4.418251,1.104563,275,'13/07/18 2:22',0.9895812,'TRUE',2.8514,-2.421932,1.803821,0.1420937,19.41035,-10.33971,4.994291,-3.19803,0.5932726,-0.2799033,3.762096,-5.902425,0.2882381,-5.905498,8.741195,-0.4245991,2.395039,-0.7456794,6.898268,-1.127218,1.703492,-5.752116,3.996971,-8.95153,10.22003,-1.595172,2.247191,-1.121793,4.996044,-6.837433,4.78311,-6.228294,0.7635236,-1.651543,3.320588,-4.662266,8.282479,-7.533159,3.42775,-5.83142,3.538565,-8.041959,5.604261,-7.598347,2.287185,-3.965398,-0.2936086,-4.733507,0.7539286,-8.021865,-0.3866819,-5.818779,1.083508,-5.754504,0.2697741,-4.017708,6.618633,-4.793797,0.9704788,-8.445375,7.017274,-4.309704],
['KY',12.76638,4.255461,4.637216,1.159304,200,'31/05/02 15:13',0.9818932,'TRUE',22.46028,9.091237,11.33573,5.214643,22.17157,14.34025,14.84693,13.49144,19.34599,9.316383,11.15997,11.23271,14.11339,12.19635,11.55645,14.43205,15.46946,14.89094,9.404753,10.79446,19.93939,3.656265,20.31931,10.68032,26.11576,13.04341,27.57416,13.69934,15.96372,8.091751,20.2443,12.82367,26.48169,14.26837,13.91523,9.974995,11.60701,15.51763,10.1793,14.61067,12.57748,5.229428,12.98358,14.08325,16.73077,11.02146,12.44928,11.49129,22.54166,15.08719,14.38561,8.998335,17.21011,7.226648,24.4104,15.72218,9.494973,5.325149,24.37264,10.20268,28.17513,10.04264],
['CT',12.21648,4.072161,4.573806,1.143452,300,'1/03/12 10:50',0.98971,'TRUE',-1.658414,-10.9939,-1.937163,-10.25981,-1.195308,-8.606098,-1.554342,-12.70384,0.2668873,-7.691629,2.69716,-7.51222,-2.9216,-7.60612,-0.5862359,-7.609216,-1.873049,-9.402542,-2.002848,-8.135921,-1.871041,-7.266486,-2.711555,-11.66985,-0.9914851,-8.3723,-2.391308,-14.49702,-2.501041,-11.94528,-3.126329,-7.522929,-1.936209,-8.830083,-2.998604,-10.97404,-2.545327,-13.96437,-2.980918,-8.183976,0.311937,-9.154572,-3.293516,-9.396478,-2.792858,-7.689294,-2.446307,-11.11649,-1.908358,-8.362305,-1.170622,-9.072707,-2.672267,-7.925601,-3.069303,-9.646969,-0.07557144,-11.04824,-3.105785,-12.43558,-2.232244,-8.368411],
['CT',9.625083,3.208361,4.705291,1.176323,275,'25/11/16 22:08',0.865485,'FALSE',-1.658414,-10.9939,-1.937163,-10.25981,-1.195308,-8.606098,-1.554342,-12.70384,0.2668873,-7.691629,2.69716,-7.51222,-2.9216,-7.60612,-0.5862359,-7.609216,-1.873049,-9.402542,-2.002848,-8.135921,-1.871041,-7.266486,-2.711555,-11.66985,-0.9914851,-8.3723,-2.391308,-14.49702,-2.501041,-11.94528,-3.126329,-7.522929,-1.936209,-8.830083,-2.998604,-10.97404,-2.545327,-13.96437,-2.980918,-8.183976,0.311937,-9.154572,-3.293516,-9.396478,-2.792858,-7.689294,-2.446307,-11.11649,-1.908358,-8.362305,-1.170622,-9.072707,-2.672267,-7.925601,-3.069303,-9.646969,-0.07557144,-11.04824,-3.105785,-12.43558,-2.232244,-8.368411],
['FL',17.85818,5.952728,4.399894,1.099974,300,'25/11/01 18:16',0.7642509,'FALSE',10.32168,5.872889,21.95105,-2.874581,6.51763,-8.621648,4.058293,3.500825,3.892226,3.077705,6.820755,2.913943,10.62536,5.937967,11.18945,6.205084,10.6432,-7.265468,3.760776,-4.693088,24.2411,-4.036252,12.14198,1.464403,12.32776,1.994331,9.493693,-1.086546,7.849817,2.205197,5.746057,4.358121,9.027902,0.1798456,16.78342,-9.455979,4.034511,-0.1012211,4.810216,-5.615338,11.73815,1.85157,3.611376,-1.954806,11.6319,3.41892,3.517238,0.5853885,4.945271,4.676959,6.96969,5.579078,17.6168,3.732666,12.09296,4.426332,6.20668,4.294805,14.51202,-3.4485,17.5475,1.991567],
['DE',17.28716,5.762388,4.283211,1.070803,300,'27/07/17 19:45',0.5777664,'FALSE',13.96078,-2.754078,17.8166,-0.2489798,11.93943,5.435248,19.21992,6.727683,10.08444,-3.310574,7.241928,7.051814,17.85883,-7.081769,14.22044,3.07837,16.71284,7.325282,19.33307,4.639416,13.89478,6.945053,6.215093,-4.174345,11.39258,2.341135,9.024965,-0.9651989,16.30217,5.354174,32.52575,5.573918,21.19589,6.123736,16.65708,5.262541,8.372107,-3.219587,7.835368,0.922116,12.77252,-2.981667,8.253579,-1.587934,8.729166,5.412447,5.57432,3.338246,13.58406,5.452766,11.59805,3.547957,11.75507,5.498366,15.33035,4.497998,3.521219,-3.946181,11.73532,-2.694235,2.978961,2.05522],
['DC',17.28162,5.76054,4.86257,1.215642,275,'16/05/01 5:59',0.9208778,'TRUE',7.469709,10.6893,18.29722,7.559317,8.489743,-3.168853,14.71674,8.316207,15.70391,8.286606,10.54364,4.470983,9.806995,3.455532,15.32086,8.475863,16.6405,-2.045821,12.9084,9.461186,12.23517,3.18436,9.566648,5.912929,8.566251,9.410266,22.97381,8.462457,30.01167,1.056327,5.733205,2.561738,15.91757,8.504434,9.987944,9.57527,8.520579,-5.626044,9.433581,8.790739,6.544751,9.22794,6.936818,4.144661,10.32389,6.937299,13.75679,-0.6973994,9.777148,2.489468,6.443116,4.349396,15.84071,2.015728,14.64257,8.044127,9.416532,6.192503,7.30854,2.94854,8.068276,4.816947],
['FL',15.12272,5.040905,4.783088,1.195772,300,'1/12/02 14:11',0.9664124,'FALSE',10.32168,5.872889,21.95105,-2.874581,6.51763,-8.621648,4.058293,3.500825,3.892226,3.077705,6.820755,2.913943,10.62536,5.937967,11.18945,6.205084,10.6432,-7.265468,3.760776,-4.693088,24.2411,-4.036252,12.14198,1.464403,12.32776,1.994331,9.493693,-1.086546,7.849817,2.205197,5.746057,4.358121,9.027902,0.1798456,16.78342,-9.455979,4.034511,-0.1012211,4.810216,-5.615338,11.73815,1.85157,3.611376,-1.954806,11.6319,3.41892,3.517238,0.5853885,4.945271,4.676959,6.96969,5.579078,17.6168,3.732666,12.09296,4.426332,6.20668,4.294805,14.51202,-3.4485,17.5475,1.991567]
]})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)
print(prediction)

{"result": [1358.0364919856179, 1165.4468685286188, 1974.4219433798783, 2060.783847563827, 1875.5603215853744, 2160.4825013262944, 1455.8311329305204, 1715.142822632055]}
CPU times: user 1.25 s, sys: 583 ms, total: 1.83 s
Wall time: 1.57 s
